# Basic HMS Workflow: Initialize, Execute, and Extract Results

This notebook demonstrates the fundamental hms-commander workflow:

1. **Initialize** an HMS project with `init_hms_project()`
2. **Execute** a simulation run with `HmsCmdr.compute_run()`
3. **Extract** results with `HmsResults`

This is the recommended starting point for new users.

In [ ]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

In [ ]:
from pathlib import Path
from hms_commander import (
    HmsExamples,
    init_hms_project,
    HmsCmdr,
    HmsResults,
    HmsDss
)

print("hms-commander loaded successfully")

## 1. Extract Example Project

We use `HmsExamples.extract_project()` to get a real HMS project from your HMS installation. This ensures reproducibility across all users.

In [ ]:
# Extract the 'tifton' example project
# This comes bundled with HEC-HMS installations
project_path = HmsExamples.extract_project(
    "tifton",
    output_path=Path.cwd() / "hms_example_projects" / "tifton_basic_workflow"
)

print(f"Project extracted to: {project_path}")

## 2. Initialize Project

The `init_hms_project()` function:
- Parses all HMS files (.hms, .basin, .met, .control, .run, .gage)
- Builds DataFrames for easy data access
- Returns an `HmsPrj` object with project metadata

In [ ]:
# Initialize the project
hms = init_hms_project(project_path)

# Display project summary
print(f"Project: {hms.project_name}")
print(f"HMS Version: {hms.hms_version}")
print(f"Basin Models: {len(hms.basin_df)}")
print(f"Met Models: {len(hms.met_df)}")
print(f"Control Specs: {len(hms.control_df)}")
print(f"Simulation Runs: {len(hms.run_df)}")

In [ ]:
# View the run configurations
print("Available Simulation Runs:")
print("=" * 60)
hms.run_df[['name', 'basin_model', 'met_model', 'control_spec', 'dss_file']]

## 3. Select a Run

Choose which simulation run to execute. The `run_df` shows all available runs with their configurations.

In [ ]:
# Select the first available run
run_name = hms.run_df['name'].iloc[0]
print(f"Selected run: '{run_name}'")

# Get the run configuration details
run_config = hms.get_run_configuration(run_name)
print("\nRun Configuration:")
for key, value in run_config.items():
    print(f"  {key}: {value}")

## 4. Execute the Simulation

`HmsCmdr.compute_run()` executes the HMS simulation:
- Generates a Jython script
- Launches HEC-HMS in headless mode
- Writes results to DSS file
- Returns `True` if successful, `False` otherwise

**Note**: Requires HEC-HMS 4.x installed on your system.

In [ ]:
# Execute the simulation run
print(f"Executing run: '{run_name}'...")
print("=" * 50)

success = HmsCmdr.compute_run(run_name, hms_object=hms)

if success:
    print(f"\nRun completed successfully!")
else:
    print(f"\nRun failed or completed with warnings.")
    print("Check console output above for details.")

## 5. Locate Results DSS File

After execution, results are written to a DSS file. The location is specified in the run configuration.

In [ ]:
# Get the DSS result file path
dss_file = hms.get_run_dss_file(run_name)

print(f"DSS File: {dss_file}")
print(f"File exists: {dss_file.exists() if dss_file else False}")

## 6. Extract Peak Flows

`HmsResults.get_peak_flows()` extracts peak discharge values for all elements in the DSS file.

In [ ]:
# Check if DSS operations are available
if HmsDss.is_available() and dss_file and dss_file.exists():
    # Extract peak flows
    peaks = HmsResults.get_peak_flows(dss_file)
    
    print("Peak Flows:")
    print("=" * 60)
    display(peaks)
    
    # Validate: peaks should be positive
    if not peaks.empty:
        assert (peaks['Peak Flow (cfs)'] > 0).all(), "All peak flows should be positive"
        print("\n[OK] All peak flows are positive")
else:
    print("DSS operations not available or results file not found.")
    print("Install pyjnius and ensure Java is available for DSS support.")

## 7. Extract Hydrograph Time Series

`HmsResults.get_outflow_timeseries()` extracts the full hydrograph for a specific element.

In [ ]:
if HmsDss.is_available() and dss_file and dss_file.exists():
    # Get the outlet element name from the basin
    # For tifton, this is typically the downstream-most junction
    
    # First, list available pathnames in DSS
    catalog = HmsDss.get_catalog(dss_file)
    flow_paths = [p for p in catalog if 'FLOW' in p.upper()]
    
    print(f"Found {len(flow_paths)} flow time series in DSS file")
    print("\nSample paths:")
    for path in flow_paths[:5]:
        print(f"  {path}")
else:
    print("DSS operations not available")

In [ ]:
if HmsDss.is_available() and dss_file and dss_file.exists() and flow_paths:
    # Read the first flow time series
    pathname = flow_paths[0]
    
    print(f"Reading: {pathname}")
    flows = HmsDss.read_timeseries(dss_file, pathname)
    
    print(f"\nTime series shape: {flows.shape}")
    print(f"Time range: {flows.index.min()} to {flows.index.max()}")
    print(f"Peak flow: {flows['value'].max():.1f} {flows.attrs.get('units', 'CFS')}")
    print(f"\nFirst 10 values:")
    display(flows.head(10))

## 8. Visualize the Hydrograph (Optional)

Plot the flow hydrograph using matplotlib.

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    if 'flows' in dir() and flows is not None and not flows.empty:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        ax.plot(flows.index, flows['value'], 'b-', linewidth=1.5)
        ax.fill_between(flows.index, flows['value'], alpha=0.3)
        
        ax.set_xlabel('Time')
        ax.set_ylabel(f"Flow ({flows.attrs.get('units', 'CFS')})")
        ax.set_title(f"Hydrograph - {run_name}")
        ax.grid(True, alpha=0.3)
        
        # Mark peak
        peak_idx = flows['value'].idxmax()
        peak_val = flows['value'].max()
        ax.axhline(y=peak_val, color='r', linestyle='--', alpha=0.5, label=f'Peak: {peak_val:.1f}')
        ax.legend()
        
        plt.tight_layout()
        plt.show()
    else:
        print("No flow data available for plotting")
        
except ImportError:
    print("matplotlib not installed. Install with: pip install matplotlib")

## Summary

This notebook demonstrated the core hms-commander workflow:

| Step | Function | Description |
|------|----------|-------------|
| 1 | `HmsExamples.extract_project()` | Get reproducible example project |
| 2 | `init_hms_project()` | Initialize project and build DataFrames |
| 3 | `hms.run_df` | Discover available simulation runs |
| 4 | `HmsCmdr.compute_run()` | Execute HMS simulation (returns bool) |
| 5 | `HmsResults.get_peak_flows()` | Extract peak flow summary |
| 6 | `HmsDss.read_timeseries()` | Extract full hydrograph |

## Next Steps

- **02_project_dataframes.ipynb**: Explore project structure via DataFrames
- **03_file_ops_basin_met_control_gage.ipynb**: Work with individual HMS files
- **04_run_management.ipynb**: Configure and validate simulation runs